# supervised model 

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

from xgboost import XGBClassifier


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

from xgboost import XGBClassifier


In [ ]:
df = pd.read_csv("bell_dns.csv")
df = df.dropna().reset_index(drop=True)


In [ ]:
df["byte_ratio"] = df["total_fwd_bytes"] / (df["total_bwd_bytes"] + 1)
df["pkt_ratio"]  = df["fwd_pkt_len_mean"] / (df["bwd_pkt_len_mean"] + 1)
df["flow_rate"]  = df["flow_duration"] / (df["tot_fwd_pkts"] + 1)


In [ ]:
X = df.drop("label", axis=1)
y = df["label"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)


In [ ]:
rf = RandomForestClassifier(n_estimators=150, random_state=42)
rf.fit(X_train, y_train)

rf_pred = rf.predict(X_test)
print(classification_report(y_test, rf_pred))


In [ ]:
xgb = XGBClassifier(eval_metric="logloss")
xgb.fit(X_train, y_train)

xgb_pred = xgb.predict(X_test)
print(classification_report(y_test, xgb_pred))

# Probability output for hybrid model
psup = xgb.predict_proba(X_scaled)[:, 1]


In [ ]:
print("ROC-AUC:", roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1]))


In [ ]:
import pandas as pd

pd.DataFrame({
    "psup": psup
}).to_csv("../outputs/psup.csv", index=False)
